In [1]:
!pip install -q --upgrade transformers datasets accelerate
!pip install -q PyGithub
# !pip install -q top  # Uncomment if you need it


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 95.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 M

In [2]:
import transformers
print(transformers.__version__)
# Should print 4.8.0 or higher


4.53.3


In [ ]:
!pip install -q PyGithub datasets

from github import Github
import re
from datasets import Dataset
import os

# Step 1: Add your GitHub token here directly or via Kaggle Secrets (more secure)
GITHUB_TOKEN = ""  # Replace with your actual token

# Step 2: Initialize PyGithub with token
g = Github(GITHUB_TOKEN)

# Step 3: Specify the GitHub repo you want to fetch from
repo = g.get_repo("openai/gym")

# Step 4: Function to extract Python function names using regex
def extract_functions_from_code(code):
    pattern = re.compile(r"def\s+(\w+)\s*\(.*\):")
    functions = pattern.findall(code)
    return functions

# Step 5: Recursively get all .py files from repo
python_files = []
contents = repo.get_contents("")
while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))
    elif file_content.path.endswith(".py"):
        python_files.append(file_content)

# Step 6: Extract function names and corresponding full code
data = {"code": [], "function_name": []}
for file in python_files:
    try:
        code = file.decoded_content.decode("utf-8")
        functions = extract_functions_from_code(code)
        for function in functions:
            data["code"].append(code)
            data["function_name"].append(function)
    except Exception as e:
        print(f"Error in file {file.path}: {e}")

# Step 7: Create Hugging Face Dataset
dataset = Dataset.from_dict(data)

# Step 8: Save the dataset to disk (in Kaggle working dir)
output_path = "/kaggle/working/code_generation_dataset"
dataset.save_to_disk(output_path)

print("✅ Dataset created and saved to:", output_path)


Saving the dataset (0/1 shards):   0%|          | 0/974 [00:00<?, ? examples/s]

✅ Dataset created and saved to: /kaggle/working/code_generation_dataset


In [ ]:
import transformers
print(transformers.__version__)


In [4]:
# ── Cell 2: Imports & Version Check (optional) ─────────────────────────────────
import transformers, datasets, accelerate
print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)
print("accelerate:", accelerate.__version__)

from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)

# ── Cell 3: Load Tokenizer & Model ────────────────────────────────────────────
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
model     = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")

# ensure padding token is defined
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# ── Cell 4: Load & Split Dataset ──────────────────────────────────────────────
dataset = load_from_disk("/kaggle/working/code_generation_dataset")
dataset = dataset.train_test_split(test_size=0.1)

# ── Cell 5: Preprocessing Function (with labels) ──────────────────────────────
def preprocess_function(examples):
    # tokenize
    model_inputs = tokenizer(
        examples["code"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    # for causal LM, labels = input IDs
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

# ── Cell 6: Tokenize & Remove Unused Columns ─────────────────────────────────
tokenized = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["code", "function_name"]
)

# ── Cell 7: Define TrainingArguments & Trainer ───────────────────────────────
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=1_000,
    save_total_limit=2,
    logging_dir="/kaggle/working/logs",
    logging_steps=500,
    do_train=True,
    do_eval=True,
    eval_steps=1_000,
    report_to="none"        # disable W&B prompts
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer   # you can replace with processing_class=tokenizer to silence the deprecation warning
)

# ── Cell 8: Train ─────────────────────────────────────────────────────────────
trainer.train()

transformers: 4.53.3
datasets: 4.0.0
accelerate: 1.9.0


2025-07-23 08:23:36.522473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753259016.705199      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753259016.760864      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/876 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

/tmp/ipykernel_36/1573184496.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=438, training_loss=0.2835505237318065, metrics={'train_runtime': 197.1321, 'train_samples_per_second': 4.444, 'train_steps_per_second': 2.222, 'total_flos': 818849205190656.0, 'train_loss': 0.2835505237318065, 'epoch': 1.0})

In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Path to the fine-tuned model checkpoint (folder containing config, pytorch_model.bin, tokenizer files)
model_path = "/kaggle/working/results/checkpoint-438"  # Change if using locally

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Text generation function
def generate_code(prompt, max_length=350):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            num_return_sequences=1
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove prompt from generated output if present
    return generated[len(prompt):].strip() if generated.startswith(prompt) else generated

# Ask user for prompt input
user_prompt = input("Enter your Python function prompt (e.g., 'def merge_sort(arr):'): ")

print("=" * 60)
print(f"🧠 Prompt: {user_prompt}")
print(generate_code(user_prompt))
print("=" * 60)


Enter your Python function prompt (e.g., 'def merge_sort(arr):'):  def binary_search(arr,n):


🧠 Prompt: def binary_search(arr,n):
l = 0
    r = n-1
    while l <= r:
        m = l + (r - l) // 2
        if arr[m] == n:
            return m
        if arr[m] > n:
            r = m - 1
        else:
            l = m + 1
    return -1

def count_sort(arr):
    return counting_sort(arr,0,len(arr))

def testing_binary_search():
    for i in range(100):
        arr = random.sample(range(100), k=5)
        n = random.randint(1, 100)
        result = binary_search(arr, n)
        assert result in arr

def testing_count_sort():
    for i in range(100):
        arr = random.sample(range(100), k=5)
        n = random.randint(1, 100)
        result = count_sort(arr, n)
        assert result in arr


def main():
    arr = list(range(100))
    n = random.randint(1, 100)
    result = binary_search(arr, n)
    assert result in arr

    arr = [random.randint(0, 10) for _ in range(100)]
    n = random.randint(1, 10)
    result = count_sort(arr, n)
    assert result in arr


def test_count_sort(